# Builders

Domain-specific languages for structured output: HTML, Markdown, XML with validation.

In [ ]:
# Setup: Install genro-bag (run this cell first on Colab)\n!pip install -q genro-bag

In [ ]:
from genro_bag import Bag, BagBuilderBase
from genro_bag.builders import element

## The Core Idea

Builders define a **vocabulary** of valid elements. When you build with a builder, you get:

- Method chaining for nested structures
- Structure validation (what can contain what)
- Semantic tags on nodes

In [ ]:
# A minimal builder for documents
class DocBuilder(BagBuilderBase):
    """Builder for simple documents."""
    
    @element(sub_tags="section, paragraph")
    def document(self): ...
    
    @element(sub_tags="paragraph, heading")
    def section(self): ...
    
    @element()
    def heading(self): ...
    
    @element()
    def paragraph(self): ...

In [ ]:
# Use the builder
store = Bag(builder=DocBuilder)

doc = store.document()
sec = doc.section(title="Introduction")
sec.heading("Welcome")
sec.paragraph("This is the first paragraph.")
sec.paragraph("This is the second paragraph.")

# Inspect the structure
for node in store.walk():
    indent = "  " * node.path.count(".")
    print(f"{indent}{node.tag}: {node.value if not isinstance(node.value, Bag) else ''}")

## Structure Validation

The `sub_tags` parameter defines what elements can be nested inside others.

In [ ]:
# Check structure validity
errors = store.builder.check()
if errors:
    for path, node, reasons in errors:
        print(f"{path}: {reasons}")
else:
    print("Structure is valid!")

In [ ]:
# Try an invalid structure
invalid_store = Bag(builder=DocBuilder)
doc = invalid_store.document()
doc.heading("Direct heading")  # heading is not allowed directly in document!

errors = invalid_store.builder.check()
for path, node, reasons in errors:
    print(f"Error at {path}: {reasons}")

## Cardinality Constraints

Control how many of each element type can appear.

In [ ]:
class FormBuilder(BagBuilderBase):
    """Builder with cardinality constraints."""
    
    @element(sub_tags="header[:1], field[1:], submit[:1]")
    def form(self): ...
    
    @element()
    def header(self): ...  # At most 1
    
    @element()
    def field(self): ...   # At least 1
    
    @element()
    def submit(self): ...  # At most 1

In [ ]:
# Valid form
store = Bag(builder=FormBuilder)
form = store.form()
form.header("User Registration")
form.field(name="username", type="text")
form.field(name="email", type="email")
form.submit("Register")

errors = store.builder.check()
print(f"Valid: {len(errors) == 0}")

In [ ]:
# Invalid: missing required field
store = Bag(builder=FormBuilder)
form = store.form()
form.header("Empty Form")
# No fields added!

errors = store.builder.check()
for path, node, reasons in errors:
    print(f"{path}: {reasons}")

## Attributes on Elements

Pass keyword arguments to set attributes on nodes.

In [ ]:
store = Bag(builder=DocBuilder)
doc = store.document(author="Alice", version="1.0")
sec = doc.section(title="Chapter 1", level=1)
sec.paragraph("Content here.", style="italic")

# Access attributes
doc_node = store.get_node('document_0')
print(f"Author: {doc_node.attr.get('author')}")
print(f"Version: {doc_node.attr.get('version')}")

## Built-in Builders

genro-bag includes builders for common formats.

In [ ]:
from genro_bag.builders import HtmlBuilder

store = Bag(builder=HtmlBuilder)
html = store.html()
head = html.head()
head.title("My Page")

body = html.body()
body.h1("Welcome")
body.p("This is a paragraph.")

div = body.div(class_="container")
div.p("Inside the div.")

print("Structure created successfully!")

In [ ]:
from genro_bag.builders import MarkdownBuilder

store = Bag(builder=MarkdownBuilder)
doc = store.document()
doc.h1("Title")
doc.paragraph("Some text.")
doc.h2("Subtitle")
doc.paragraph("More text.")

# Generate markdown output
# md_text = store.builder.render()
print("Markdown structure created!")

## Key Takeaways

- **Vocabulary**: Define what elements exist with `@element`
- **Structure**: Control nesting with `sub_tags`
- **Cardinality**: Constrain counts with `[:1]`, `[1:]`, etc.
- **Validation**: Call `builder.check()` to verify structure
- **Attributes**: Pass kwargs to set node attributes